<a href="https://colab.research.google.com/github/alscop/ESAA-25-2/blob/main/%EA%B3%B5%EB%8F%99_%EC%BD%94%EB%93%9C_1%EC%8B%9C%EA%B0%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
file_path_train = '/content/drive/MyDrive/BDA/DATA/train.csv'
chunk_size = 500000
result_list_train = []

# 청크로 읽으면서 즉시 최적화
for chunk in pd.read_csv(file_path_train, chunksize=chunk_size):
    # 실수형 최적화
    fcols = chunk.select_dtypes('float').columns
    chunk[fcols] = chunk[fcols].apply(pd.to_numeric, downcast='float')

    # 정수형 최적화
    icols = chunk.select_dtypes('integer').columns
    chunk[icols] = chunk[icols].apply(pd.to_numeric, downcast='integer')

    # 카테고리형 변환
    chunk['module(equipment)'] = chunk['module(equipment)'].astype('category')

    result_list_train.append(chunk)

# 하나로 합치기
train = pd.concat(result_list_train, ignore_index=True)

In [4]:
file_path_test = '/content/drive/MyDrive/BDA/DATA/test.csv' # 실제 파일명 확인
chunk_size = 500000  # 10만보다 조금 더 크게 잡아도 효율적입니다.
result_list2 = []

# 1. 청크 단위로 읽기 시작
# 한글 깨짐 방지를 위해 encoding='cp949' 추가
for chunk in pd.read_csv(file_path_test, chunksize=chunk_size, encoding='cp949'):

    # 2. 각 청크별로 데이터 타입 즉시 최적화 (Downcasting)
    for col in chunk.columns:
        col_type = chunk[col].dtype

        if col_type == 'object':
            # module(equipment) 같은 문자열은 카테고리형으로
            chunk[col] = chunk[col].astype('category')
        elif str(col_type)[:3] == 'int':
            # 정수형은 더 작은 int 타입으로
            chunk[col] = pd.to_numeric(chunk[col], downcast='integer')
        elif str(col_type)[:5] == 'float':
            # 실수형(float64)은 float32로
            chunk[col] = pd.to_numeric(chunk[col], downcast='float')

    result_list2.append(chunk)

# 3. 최적화된 청크들을 하나로 합치기
test = pd.concat(result_list2, ignore_index=True)

In [5]:
# module 컬럼을 카테고리로 변경 (메모리 줄이기)
train['module(equipment)'] = train['module(equipment)'].astype('category')
test['module(equipment)'] = test['module(equipment)'].astype('category')

# 결과 확인
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27181440 entries, 0 to 27181439
Data columns (total 20 columns):
 #   Column                Dtype   
---  ------                -----   
 0   module(equipment)     category
 1   timestamp             int64   
 2   localtime             int64   
 3   operation             int8    
 4   voltageR              float32 
 5   voltageS              float32 
 6   voltageT              float32 
 7   voltageRS             float32 
 8   voltageST             float32 
 9   voltageTR             float32 
 10  currentR              float32 
 11  currentS              float32 
 12  currentT              float32 
 13  activePower           float32 
 14  powerFactorR          float32 
 15  powerFactorS          float32 
 16  powerFactorT          float32 
 17  reactivePowerLagging  float32 
 18  accumActiveEnergy     int32   
 19  datetime              object  
dtypes: category(1), float32(14), int32(1), int64(2), int8(1), object(1)
memory usage: 2.2+ G

datetime 변환

In [6]:
import gc

# 수학적 연산 (몫과 나머지)을 이용한 피처 추출
# 이 방식은 메모리 복사본을 만들지 않아 가장 안전합니다.
# localtime이 20241201083000 (int64) 형태일 때:
train['month']   = ((train['localtime'] // 100000000) % 100).astype('int8')
train['day']     = ((train['localtime'] // 1000000) % 100).astype('int8')
train['hour']    = ((train['localtime'] // 10000) % 100).astype('int8')
train['minute']  = ((train['localtime'] // 100) % 100).astype('int8')

# 요일(weekday) 계산 - 3,300만 건을 다 바꾸지 않고 날짜별로 매핑 (RAM 보호 핵심)
print("요일 정보 매핑 중...")
unique_days = (train['localtime'] // 1000000).unique() # YYYYMMDD 고유값만 추출
day_to_weekday = {d: pd.to_datetime(str(d), format='%Y%m%d').weekday() for d in unique_days}
train['weekday'] = (train['localtime'] // 1000000).map(day_to_weekday).astype('int8')

# 4메모리 최적화 (float64 -> float32)
# 모든 컬럼을 유지하려면 각 컬럼의 크기를 줄이는 것이 필수입니다.
print("수치형 컬럼 최적화 중...")
fcols = train.select_dtypes('float64').columns
train[fcols] = train[fcols].astype('float32')

icols = train.select_dtypes('int64').columns
for col in icols:
    if col != 'localtime': # localtime은 연산을 위해 유지하거나 나중에 삭제
        train[col] = pd.to_numeric(train[col], downcast='integer')

gc.collect()

# 최종 확인
print("\n [완료] 모든 컬럼 유지 및 시간 분리 성공")
print(f"현재 전체 컬럼 수: {len(train.columns)}개")
print(f"hour 고유값 개수: {train['hour'].nunique()}개")
display(train[['localtime', 'month', 'day', 'hour', 'weekday']].tail(3))

요일 정보 매핑 중...
수치형 컬럼 최적화 중...

 [완료] 모든 컬럼 유지 및 시간 분리 성공
현재 전체 컬럼 수: 25개
hour 고유값 개수: 24개


,localtime,month,day,hour,weekday
27181437,20250331235945,3,31,23,0
27181438,20250331235950,3,31,23,0
27181439,20250331235955,3,31,23,0


In [7]:
# 1. 시간 범위와 고유값 개수 확인 (24개여야 함)
print(f"Hour 고유값 ({train['hour'].nunique()}개): {sorted(train['hour'].unique())}")
print(f"Month 고유값: {sorted(train['month'].unique())}")

# 2. 요일이 잘 들어갔는지 확인 (0:월 ~ 6:일)
print(f"Weekday 고유값: {sorted(train['weekday'].unique())}")

# 3. Head와 Tail을 동시에 찍어서 시간이 흐르는지 직접 대조
print("\n [데이터 시작점 vs 끝점 대조]")
display(train[['month', 'day', 'hour', 'minute', 'weekday']].iloc[[0, -1]])

Hour 고유값 (24개): [np.int8(0), np.int8(1), np.int8(2), np.int8(3), np.int8(4), np.int8(5), np.int8(6), np.int8(7), np.int8(8), np.int8(9), np.int8(10), np.int8(11), np.int8(12), np.int8(13), np.int8(14), np.int8(15), np.int8(16), np.int8(17), np.int8(18), np.int8(19), np.int8(20), np.int8(21), np.int8(22), np.int8(23)]
Month 고유값: [np.int8(1), np.int8(2), np.int8(3), np.int8(12)]
Weekday 고유값: [np.int8(0), np.int8(1), np.int8(2), np.int8(3), np.int8(4), np.int8(5), np.int8(6)]

 [데이터 시작점 vs 끝점 대조]


,month,day,hour,minute,weekday
0,12,1,0,0,6
27181439,3,31,23,59,0


In [8]:
# 1. 형식을 판다스가 알아서 추론하여 datetime 객체로 변환
temp_test_dt = pd.to_datetime(test['datetime'], errors='coerce')

# 2. 개별 숫자 피처 추출
test['month']   = temp_test_dt.dt.month.astype('int8')
test['day']     = temp_test_dt.dt.day.astype('int8')
test['hour']    = temp_test_dt.dt.hour.astype('int8')
test['minute']  = temp_test_dt.dt.minute.astype('int8')
test['weekday'] = temp_test_dt.dt.weekday.astype('int8')

# 3. 원본 제거 및 메모리 정리
test.drop(columns=['datetime'], inplace=True, errors='ignore')

# 만약 변환 과정에서 에러가 나서 NaN이 생겼는지 확인
if test['hour'].isnull().any():
    print("경고: 일부 날짜 데이터가 변환되지 않았습니다. 데이터를 확인해 주세요.")
else:
    print("Test 데이터 변환 성공!")

del temp_test_dt
gc.collect()

# 결과 확인
display(test[['month', 'day', 'hour', 'weekday']].head(3))

Test 데이터 변환 성공!


,month,day,hour,weekday
0,4,1,0,1
1,4,1,0,1
2,4,1,0,1


In [9]:
# 1. 시간 범위와 고유값 개수 확인
print("[Test 데이터 피처 검증]")
print("-" * 40)
print(f"Hour 고유값 ({test['hour'].nunique()}개): {sorted(test['hour'].unique())}")
print(f"Month 고유값: {sorted(test['month'].unique())}")
print(f"Weekday 고유값: {sorted(test['weekday'].unique())}")

# 2. 데이터의 시작(Head)과 끝(Tail) 확인
print("\n[Test 시작점 vs 끝점 대조]")
print("-" * 40)
# 첫 3행과 마지막 3행을 동시에 출력
display(test[['month', 'day', 'hour', 'minute', 'weekday']].iloc[[0, 1, 2, -3, -2, -1]])

# 3. 결측치 확인 (변환 과정에서 실패한 데이터가 있는지)
null_count = test[['month', 'day', 'hour', 'minute', 'weekday']].isnull().sum().sum()
if null_count == 0:
    print("\n 모든 데이터가 결측치 없이 완벽하게 변환되었습니다.")
else:
    print(f"\n 주의: {null_count}개의 데이터가 변환되지 않았습니다.")

[Test 데이터 피처 검증]
----------------------------------------
Hour 고유값 (24개): [np.int8(0), np.int8(1), np.int8(2), np.int8(3), np.int8(4), np.int8(5), np.int8(6), np.int8(7), np.int8(8), np.int8(9), np.int8(10), np.int8(11), np.int8(12), np.int8(13), np.int8(14), np.int8(15), np.int8(16), np.int8(17), np.int8(18), np.int8(19), np.int8(20), np.int8(21), np.int8(22), np.int8(23)]
Month 고유값: [np.int8(4)]
Weekday 고유값: [np.int8(0), np.int8(1), np.int8(2), np.int8(3), np.int8(4), np.int8(5), np.int8(6)]

[Test 시작점 vs 끝점 대조]
----------------------------------------


,month,day,hour,minute,weekday
0,4,1,0,0,1
1,4,1,0,0,1
2,4,1,0,0,1
1048572,4,3,16,20,3
1048573,4,3,16,20,3
1048574,4,3,16,21,3



 모든 데이터가 결측치 없이 완벽하게 변환되었습니다.


## 1시간 단위로 데이터 변환한 부분 적기

이서현

In [10]:
import pandas as pd
import gc

def process_B_part_1hour(df):
    target_cols = ['operation', 'activePower']
    resampled_chunks = []

    for mod in df['module(equipment)'].unique():
        # 1. 1시간 단위(1H) mean, std, max 동시 계산
        temp = df[df['module(equipment)'] == mod][['timestamp', 'module(equipment)', 'accumActiveEnergy'] + target_cols].copy()
        temp['datetime'] = pd.to_datetime(temp['timestamp'], unit='ms')
        temp = temp.set_index('datetime')

        # 1시간 단위는 변동폭이 크므로 max 값을 추가하는 것이 모델 성능에 좋습니다.
        res = temp[target_cols].resample('1H').agg(['mean', 'std', 'max'])

        # 2. 통합 지표 생성 (멀티인덱스 처리)
        # 컬럼명을 'activePower_mean_1h' 형태로 명확하게 구분
        res.columns = [f"{c[0]}_{c[1]}_1h" for c in res.columns]

        # 해당 1시간 동안의 순수 에너지 소비량(Delta) 계산
        res['energy_delta_1h'] = temp['accumActiveEnergy'].resample('1H').apply(
            lambda x: x.max() - x.min() if len(x) > 0 else 0
        )

        # 3. 결과 정리
        res = res.reset_index()
        res['module'] = mod
        resampled_chunks.append(res)

        # 4. 사용 완료된 메모리 즉시 해제
        del temp
        gc.collect()
        print(f"✅ {mod} 1시간 단위 처리 완료")

    return pd.concat(resampled_chunks, axis=0)

# 실행 및 확인
df_1hour_B = process_B_part_1hour(train)
display(df_1hour_B.head())

/tmp/ipython-input-1122270590.py:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  res = temp[target_cols].resample('1H').agg(['mean', 'std', 'max'])
/tmp/ipython-input-1122270590.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  res['energy_delta_1h'] = temp['accumActiveEnergy'].resample('1H').apply(


✅ 1(PM-3) 1시간 단위 처리 완료


/tmp/ipython-input-1122270590.py:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  res = temp[target_cols].resample('1H').agg(['mean', 'std', 'max'])
/tmp/ipython-input-1122270590.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  res['energy_delta_1h'] = temp['accumActiveEnergy'].resample('1H').apply(


✅ 11(우측분전반1) 1시간 단위 처리 완료


/tmp/ipython-input-1122270590.py:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  res = temp[target_cols].resample('1H').agg(['mean', 'std', 'max'])
/tmp/ipython-input-1122270590.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  res['energy_delta_1h'] = temp['accumActiveEnergy'].resample('1H').apply(


...

/tmp/ipython-input-1122270590.py:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  res = temp[target_cols].resample('1H').agg(['mean', 'std', 'max'])
/tmp/ipython-input-1122270590.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  res['energy_delta_1h'] = temp['accumActiveEnergy'].resample('1H').apply(


✅ 4(분쇄기(1)) 1시간 단위 처리 완료


/tmp/ipython-input-1122270590.py:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  res = temp[target_cols].resample('1H').agg(['mean', 'std', 'max'])
/tmp/ipython-input-1122270590.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  res['energy_delta_1h'] = temp['accumActiveEnergy'].resample('1H').apply(


✅ 5(좌측분전반) 1시간 단위 처리 완료


,datetime,operation_mean_1h,operation_std_1h,operation_max_1h,activePower_mean_1h,activePower_std_1h,activePower_max_1h,energy_delta_1h,module
0,2024-12-01 08:00:00,1.0,0.0,1,3011.903076,738.202087,5013.270020,3007,1(PM-3)
1,2024-12-01 09:00:00,1.0,0.0,1,3011.384766,736.284058,5096.310059,3008,1(PM-3)
2,2024-12-01 10:00:00,1.0,0.0,1,2990.481934,710.641418,4971.149902,2986,1(PM-3)
3,2024-12-01 11:00:00,1.0,0.0,1,3003.309082,721.679565,4855.109863,3001,1(PM-3)
4,2024-12-01 12:00:00,1.0,0.0,1,3007.081299,709.555115,4913.540039,3003,1(PM-3)


In [11]:
# 1. 행 개수 추출
raw_count = len(train)
hour_count = len(df_1hour_B)

# 2. 통계 계산
reduction_count = raw_count - hour_count
reduction_percent = (reduction_count / raw_count) * 100
compression_factor = raw_count / hour_count

print(f"📊 [데이터 압축 리포트: 5초 → 1시간]")
print(f"· 원본 데이터 (5초 단위): {raw_count:,} 행")
print(f"· 변환 후 데이터 (1시간 단위): {hour_count:,} 행")
print("-" * 40)
print(f"· 제거된 행 개수: {reduction_count:,} 행")
print(f"· 압축률: {reduction_percent:.2f}% 감소")
print(f"· 효율성: 약 {compression_factor:.1f}배 더 가벼워짐")

📊 [데이터 압축 리포트: 5초 → 1시간]
· 원본 데이터 (5초 단위): 27,181,440 행
· 변환 후 데이터 (1시간 단위): 37,739 행
----------------------------------------
· 제거된 행 개수: 27,143,701 행
· 압축률: 99.86% 감소
· 효율성: 약 720.2배 더 가벼워짐


## 관련 변수 처리 적는 곳

#이서현

변동계수(CV, Coefficient of Variation) 추출
- 평균 대비 표준편차의 비율 (CV = 표준편차 / 평균)
- 설비마다 전력 소비 규모(예: 100kW 소형 vs 5000kW 대형)가 다르기 때문에 발생할 수 있는 규모의 왜곡을 방지하기 위함.
- 절대적인 요동 정도(std)가 아닌, 상대적인 '변동성'을 동일한 기준으로 비교함.
- 설비의 작업 모드 전환이나 공회전 여부를 판단하는 강력한 힌트로 활용됨.
- 분모에 1e-6을 더하여 평균이 0인 경우(설비 정지) 발생하는 ZeroDivisionError 방지.

In [12]:
# 1시간 버전
volatility_df_1h = train.groupby(['module(equipment)', 'month', 'day', 'hour', 'weekday'], observed=True)['activePower'].agg([
    ('p_mean_1h', 'mean'),
    ('p_std_1h', 'std')
]).reset_index()

# CV 계산 (1시간 단위의 전력 성격)
volatility_df_1h['p_cv_1h'] = volatility_df_1h['p_std_1h'] / (volatility_df_1h['p_mean_1h'] + 1e-6)

print("✅ [1시간 전용] 변동성 지표 추출 완료 (month, weekday 포함)")

✅ [1시간 전용] 변동성 지표 추출 완료 (month, weekday 포함)


- is_optimal_pf: 현재 역률이 사용자가 찾아낸 최적 구간인 70~80% 사이에 있는지 여부를 나타내는 이진 변수.

- peak_efficiency_match: 전력이 4,000kW를 초과하는 고부하 상태이면서 동시에 역률이 최적 구간(70~80%)일 때 1이 되는 변수.

- is_startup_event: 전력 부하가 4,000kW 이하에서 초과로 전환되는 기동(Startup) 순간을 나타냄.

- pf_delta_70: 현재 역률이 최적점인 70%에서 물리적으로 얼마나 떨어져 있는지 그 거리값을 나타냄.

- rolling_peak_density: 최근 1시간(720개 행) 동안 발생한 기동 이벤트의 평균 빈도로, 현재 작업이 얼마나 밀도 있게 진행 중인지를 나타냄.

In [13]:
def process_B_features_1hour(df):
    pf_cols = [c for c in df.columns if 'powerFactor' in c]
    group_keys = ['month', 'day', 'hour', 'weekday']
    resampled_chunks = []

    for mod in df['module(equipment)'].unique():
        temp = df[df['module(equipment)'] == mod].copy()
        pf_avg = temp[pf_cols].mean(axis=1).values
        p_val = temp['activePower'].values

        # 5초 단위 베이스 계산
        temp['is_optimal_pf'] = ((pf_avg >= 70) & (pf_avg <= 80)).astype(np.int8)
        is_peak = (p_val > 4000).astype(np.int8)
        temp['peak_efficiency_match'] = (is_peak & temp['is_optimal_pf'].values).astype(np.int8)
        temp['is_startup_event'] = np.concatenate([[0], (is_peak[1:] > is_peak[:-1])]).astype(np.int8)
        temp['pf_delta_70'] = np.abs(pf_avg - 70).astype(np.float32)

        # 1시간 단위로 집계
        res = temp.groupby(group_keys, observed=True).agg({
            'is_optimal_pf': 'mean',
            'peak_efficiency_match': 'mean',
            'is_startup_event': 'sum',       # 중요: 시간당 총 기동 횟수 (작업 강도)
            'pf_delta_70': 'mean'
        }).reset_index()

        # 컬럼명에 1h 접미사 추가하여 구분
        res.columns = group_keys + [f"{c}_1h" for c in res.columns if c not in group_keys]
        res['module'] = mod
        resampled_chunks.append(res)
        del temp, pf_avg, p_val; gc.collect()
        print(f"✅ {mod} 1시간 파생변수 생성 완료")

    return pd.concat(resampled_chunks, axis=0)

역전 구간

In [14]:
import numpy as np
import gc

# 1. 에너지 변화량 재계산 (보정 후 상태 반영)
energy_vals = train['accumActiveEnergy'].values
energy_diff = np.diff(energy_vals, prepend=energy_vals[0])

# 2. 물리적 모순 탐지
# (1) 에너지가 감소하는 구간 (0보다 작은 구간)
decrease_count = np.sum(energy_diff < 0)

# (2) 전력은 높은데 에너지가 전혀 안 느는 구간 (전력 > 1000인데 diff == 0)
power_vals = train['activePower'].values
stuck_count = np.sum((power_vals > 1000) & (energy_diff == 0))

print(f"📊 에너지 무결성 검사 결과:")
print(f" - 에너지가 감소하는 행(역전): {decrease_count:,} 건")
print(f" - 전력은 높은데 에너지가 멈춘 행(Stuck): {stuck_count:,} 건")

# 3. 메모리 정리
del energy_vals, energy_diff, power_vals
gc.collect()

📊 에너지 무결성 검사 결과:
 - 에너지가 감소하는 행(역전): 12 건
 - 전력은 높은데 에너지가 멈춘 행(Stuck): 1 건


0

In [15]:
# 1. 역전이 발생하는 인덱스 찾기
energy_vals = train['accumActiveEnergy'].values
energy_diff = np.diff(energy_vals, prepend=energy_vals[0])
reverse_idx = np.where(energy_diff < 0)[0]

# 2. 그 12건의 앞뒤 행(context)을 출력해서 비교하기
for idx in reverse_idx:
    print(f"\n📍 인덱스 {idx} 지점 확인:")
    # 앞행(idx-1)과 현재행(idx)의 설비ID를 비교합니다.
    display(train.iloc[idx-1 : idx+1][['module(equipment)', 'accumActiveEnergy', 'activePower']])


📍 인덱스 2090880 지점 확인:


,module(equipment),accumActiveEnergy,activePower
2090879,1(PM-3),10696748,3687.070068
2090880,11(우측분전반1),1129004,3124.790039



📍 인덱스 4181760 지점 확인:


,module(equipment),accumActiveEnergy,activePower
4181759,11(우측분전반1),9869171,2019.380005
4181760,12(4호기),4552002,1843.920044



📍 인덱스 6272640 지점 확인:


...

,module(equipment),accumActiveEnergy,activePower
20908799,2(L-1전등),10092561,3058.459961
20908800,3(분쇄기(2)),3678004,3190.000000



📍 인덱스 22999680 지점 확인:


,module(equipment),accumActiveEnergy,activePower
22999679,3(분쇄기(2)),12417747,2701.290039
22999680,4(분쇄기(1)),2462003,2730.419922



📍 인덱스 25090560 지점 확인:


,module(equipment),accumActiveEnergy,activePower
25090559,4(분쇄기(1)),11203687,4168.930176
25090560,5(좌측분전반),2831004,3204.030029


In [16]:
import numpy as np
import gc

# 1. 사용자님이 새로 추출하신 12개의 정확한 인덱스 (행 위치)
target_indices = [
    2592002, 5184003, 7776004, 10368005, 12960006, 15552007,
    18144008, 20736009, 23328010, 25920011, 28512012, 31104013
]

# 2. 넘파이 배열로 직접 접근 (메모리 복사 없이 뷰만 생성)
# .values를 사용하면 데이터프레임의 원본 메모리를 직접 가리킵니다.
energy_ptr = train['accumActiveEnergy'].values
module_ptr = train['module(equipment)'].values

print("🚀 메모리 절약형 데이터 보정 시작...")

for idx in target_indices:
    # 안전장치: 인덱스 범위 확인
    if idx >= len(energy_ptr):
        continue

    # [핵심] 리셋 직전의 거대한 누적값
    prev_val = energy_ptr[idx-1]
    # [핵심] 리셋된 직후의 작은 값
    curr_val = energy_ptr[idx]

    # 두 값의 차이(보정치) 계산
    adjustment = prev_val - curr_val

    # 해당 모듈이 끝날 때까지 뒤에 오는 모든 행에 차이값을 더함
    # 불필요한 복사본을 만들지 않기 위해 루프를 최소화하여 직접 연산
    current_mod = module_ptr[idx]

    # 메모리 폭발을 막기 위해 전체 mask를 생성하지 않고
    # 해당 지점부터 모듈이 바뀔 때까지만 한정해서 수정
    i = idx
    while i < len(energy_ptr) and module_ptr[i] == current_mod:
        energy_ptr[i] += adjustment
        i += 1

    print(f"✅ 지점 {idx}: 보정 완료 (모듈 {current_mod})")

# 3. 변경 사항이 원본 train에 즉시 반영됨 (energy_ptr이 원본을 가리키고 있기 때문)
print("\n" + "="*40)
print("📊 최종 무결성 검사 (메모리 안전 모드)")

# 검증 과정에서도 메모리 절약을 위해 필요한 부분만 비교
rev_check = (energy_ptr[1:] < energy_ptr[:-1]) & (module_ptr[1:] == module_ptr[:-1])
error_count = np.sum(rev_check)

print(f"📍 남은 역전 건수: {error_count} 건")
print("="*40)

# 메모리 강제 정리
del rev_check
gc.collect()

🚀 메모리 절약형 데이터 보정 시작...
✅ 지점 2592002: 보정 완료 (모듈 11(우측분전반1))
✅ 지점 5184003: 보정 완료 (모듈 12(4호기))
✅ 지점 7776004: 보정 완료 (모듈 13(3호기))
✅ 지점 10368005: 보정 완료 (모듈 14(2호기))
✅ 지점 12960006: 보정 완료 (모듈 16(호이스트))
✅ 지점 15552007: 보정 완료 (모듈 17(6호기))
✅ 지점 18144008: 보정 완료 (모듈 18(우측분전반2))
✅ 지점 20736009: 보정 완료 (모듈 2(L-1전등))
✅ 지점 23328010: 보정 완료 (모듈 4(분쇄기(1)))
✅ 지점 25920011: 보정 완료 (모듈 5(좌측분전반))

📊 최종 무결성 검사 (메모리 안전 모드)
📍 남은 역전 건수: 0 건


31

편한비


## 단위 데이터 생성 (1시간)
- 전압 평균
- 전압 표준편차
- 전압 불안정성
- 시간 기반 전압 변동성

-> 4개의 단위데이터 생성

In [17]:
import pandas as pd
import numpy as np
import gc

def process_voltage_8_features(df):
    # 1. 원본에서 4대 기초 파생변수 생성 (메모리 절약형 float32)
    print("🚀 기초 4대 파생변수 연산 중...")
    v_cols = ['voltageR', 'voltageS', 'voltageT']

    # [변수 1] v_avg: 실시간 평균
    v_avg = df[v_cols].mean(axis=1).astype('float32')
    # [변수 2] v_std: 실시간 상간 불평형(표준편차)
    v_std = df[v_cols].std(axis=1).astype('float32')
    # [변수 3] v_inst: 전압 불안정 종합지표 (Range + Std)
    v_range = (df[v_cols].max(axis=1) - df[v_cols].min(axis=1)).astype('float32')
    v_inst = (v_range + v_std).astype('float32')

    # [변수 4] v_roll3: 시간 기반 전압 변동성 (Rolling 3)
    # 램 보호를 위해 transform 대신 임시 할당 후 계산
    temp_df = df[['module(equipment)', 'month', 'day', 'hour', 'minute']].copy()
    temp_df['v_avg'] = v_avg
    temp_df['v_std'] = v_std
    temp_df['v_inst'] = v_inst
    temp_df['v_range'] = v_range # 계산용

    print("🚀 Rolling 변수 생성 중...")
    temp_df['v_roll3'] = temp_df.groupby('module(equipment)')['v_range'].rolling(3, min_periods=1).mean().reset_index(level=0, drop=True).astype('float32')

    group_min = ['module(equipment)', 'month', 'day', 'hour', 'minute']
    res_1min = temp_df.groupby(group_min).agg({
        'v_avg': 'mean',
        'v_std': 'mean',
        'v_inst': 'max',   # 불안정성은 최악의 상황(max)을 보는 게 효과적
        'v_roll3': 'mean'
    }).reset_index()

    # 컬럼명 정리
    res_1min.columns = group_min + ['v_avg_1T', 'v_std_1T', 'v_inst_1T', 'v_roll3_1T']

    # 3. 1시간 단위 집계 (1분 데이터를 재요약하여 램 보호)
    print("🚀 1시간 단위 집계 중 (4개 지표)...")
    group_hour = ['module(equipment)', 'month', 'day', 'hour']
    res_1hour = res_1min.groupby(group_hour).agg({
        'v_avg_1T': 'mean',
        'v_std_1T': 'mean',
        'v_inst_1T': 'max',
        'v_roll3_1T': 'mean'
    }).reset_index()

    # 컬럼명 정리
    res_1hour.columns = group_hour + ['v_avg_1H', 'v_std_1H', 'v_inst_1H', 'v_roll3_1H']

    # 4. 메모리 정리
    del v_avg, v_std, v_range, v_inst, temp_df
    gc.collect()

    return res_1hour

# --- 실행 ---
train_v_1hour = process_voltage_8_features(train)
test_v_1hour = process_voltage_8_features(test)

print("\n✅ 총 8개의 계층적 전압 지표 생성 완료!")

display(train_v_1hour.head())

🚀 기초 4대 파생변수 연산 중...
🚀 Rolling 변수 생성 중...


/tmp/ipython-input-1483134192.py:27: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  temp_df['v_roll3'] = temp_df.groupby('module(equipment)')['v_range'].rolling(3, min_periods=1).mean().reset_index(level=0, drop=True).astype('float32')
/tmp/ipython-input-1483134192.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  res_1min = temp_df.groupby(group_min).agg({


🚀 1시간 단위 집계 중 (4개 지표)...


/tmp/ipython-input-1483134192.py:43: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  res_1hour = res_1min.groupby(group_hour).agg({


🚀 기초 4대 파생변수 연산 중...
🚀 Rolling 변수 생성 중...


/tmp/ipython-input-1483134192.py:27: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  temp_df['v_roll3'] = temp_df.groupby('module(equipment)')['v_range'].rolling(3, min_periods=1).mean().reset_index(level=0, drop=True).astype('float32')
/tmp/ipython-input-1483134192.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  res_1min = temp_df.groupby(group_min).agg({


🚀 1시간 단위 집계 중 (4개 지표)...

✅ 총 8개의 계층적 전압 지표 생성 완료!


/tmp/ipython-input-1483134192.py:43: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  res_1hour = res_1min.groupby(group_hour).agg({


,module(equipment),month,day,hour,v_avg_1H,v_std_1H,v_inst_1H,v_roll3_1H
0,1(PM-3),1,1,0,215.069763,2.663517,15.101906,5.050078
1,1(PM-3),1,1,1,215.023865,2.570591,15.080875,4.886000
2,1(PM-3),1,1,2,214.997406,2.544646,14.465050,4.847894
3,1(PM-3),1,1,3,214.908264,2.596759,15.080286,4.948546
4,1(PM-3),1,1,4,215.058990,2.580853,15.196297,4.903046


- 8개의 파생변수 생성 완료 -> 데이터에 병합

In [18]:
import gc

def memory_safe_mapping(df, agg_df, keys, suffix):
    print(f"🔗 {suffix} 변수 맵핑 시작...")

    # 1. 그룹 키를 인덱스로 설정 (찾기 속도를 위해)
    agg_df_indexed = agg_df.set_index(keys)

    # 2. 붙여줄 컬럼들만 선택
    target_cols = [c for c in agg_df.columns if c not in keys]

    # 3. 데이터프레임을 합치는 대신, 'map'이나 'join'의 원리를 이용해 한 컬럼씩 추가
    # 원본의 키 조합 생성
    temp_keys = pd.MultiIndex.from_frame(df[keys])

    for col in target_cols:
        print(f"   - {col} 붙이는 중...")
        df[col] = agg_df_indexed[col].reindex(temp_keys).values
        # 4. 루프 중간중간 메모리 비우기
        gc.collect()

    del agg_df_indexed, temp_keys
    gc.collect()
    return df

# --- 실행 (1분 단위와 1시간 단위를 순차적으로 처리) ---

# 2. 1시간 단위 데이터 맵핑
keys_hour = ['module(equipment)', 'month', 'day', 'hour']
train = memory_safe_mapping(train, train_v_1hour, keys_hour, "1시간 단위")
test = memory_safe_mapping(test, test_v_1hour, keys_hour, "1시간 단위")

print("✅ 모든 변수 결합 완료!")

🔗 1시간 단위 변수 맵핑 시작...
   - v_avg_1H 붙이는 중...
   - v_std_1H 붙이는 중...
   - v_inst_1H 붙이는 중...
   - v_roll3_1H 붙이는 중...
🔗 1시간 단위 변수 맵핑 시작...
   - v_avg_1H 붙이는 중...
   - v_std_1H 붙이는 중...
   - v_inst_1H 붙이는 중...
   - v_roll3_1H 붙이는 중...
✅ 모든 변수 결합 완료!


- 기존 상전압 변수 제거

In [19]:
import gc

# 리스트로 묶어서 하나씩 확실하게 지웁니다.
for col in ['voltageR', 'voltageS', 'voltageT']:
    if col in train.columns:
        del train[col]
        print(f"🗑️ train['{col}'] 삭제 완료")
    if col in test.columns:
        del test[col]
        print(f"🗑️ test['{col}'] 삭제 완료")
    # 컬럼 하나 지울 때마다 램 청소
    gc.collect()

print("✅ 개별 del 방식을 통해 램 손상 없이 삭제 완료!")

🗑️ train['voltageR'] 삭제 완료
🗑️ test['voltageR'] 삭제 완료
🗑️ train['voltageS'] 삭제 완료
🗑️ test['voltageS'] 삭제 완료
🗑️ train['voltageT'] 삭제 완료
🗑️ test['voltageT'] 삭제 완료
✅ 개별 del 방식을 통해 램 손상 없이 삭제 완료!


- 13번 설비의 전압 분포 정점이 타 설비와 분리되어 있음 -> 13번 설비가 고유한 전압 특성을 가진 독립된 군집임

**결론**
- 13번 설비 특이점 변수(Flag) 만들기

In [20]:
import numpy as np

# 1. '저전압 위험 설비' 여부 (13번 설비 타겟팅)
# 13번 설비의 정확한 이름을 찾아 1, 아니면 0을 부여합니다.
target_mod = [m for m in train['module(equipment)'].unique() if '13(3호기)' in str(m)][0]

for df in [train, test]:
    # 설비 자체가 13번인지 여부 (카테고리 힌트)
    df['is_low_voltage_module'] = (df['module(equipment)'] == target_mod).astype(int)

    # 2. 임계치 기반 저전압 플래그
    # 박스플롯에서 확인한 13번의 하위 25% 지점 이하로 내려가면 1
    # 이 값은 실제 데이터를 보고 조정하면 더 좋습니다.
    df['voltage_under_alert'] = (df['v_avg_1H'] < 214.9).astype(int)

print("✅ 특이점 타겟팅 변수 2종 추가 완료!")

✅ 특이점 타겟팅 변수 2종 추가 완료!


- 병합 후 생긴 전압 관련 NaN 값 처리하기

In [21]:
def safe_fill_na(df, cols):
    print(f"🧹 {len(cols)}개 컬럼 결측치 순차 채우기 시작...")
    for col in cols:
        if col in df.columns:
            # 원본을 직접 수정하여 복사본 생성을 최소화
            df[col] = df[col].ffill()
            df[col] = df[col].bfill()
            # 중간중간 가비지 컬렉션
            gc.collect()
            print(f"   - {col} 완료")
    print("✅ 모든 결측치 처리 완료!")

# 전압 관련 컬럼 리스트 추출
voltage_cols = [c for c in train.columns if 'v_' in c or 'voltage' in c]

# 실행
safe_fill_na(train, voltage_cols)
safe_fill_na(test, voltage_cols)

🧹 9개 컬럼 결측치 순차 채우기 시작...
   - voltageRS 완료
   - voltageST 완료
   - voltageTR 완료
   - v_avg_1H 완료
   - v_std_1H 완료
   - v_inst_1H 완료
   - v_roll3_1H 완료
   - is_low_voltage_module 완료
   - voltage_under_alert 완료
✅ 모든 결측치 처리 완료!
🧹 9개 컬럼 결측치 순차 채우기 시작...
   - voltageRS 완료
   - voltageST 완료
   - voltageTR 완료
   - v_avg_1H 완료
   - v_std_1H 완료
   - v_inst_1H 완료
   - v_roll3_1H 완료
   - is_low_voltage_module 완료
   - voltage_under_alert 완료
✅ 모든 결측치 처리 완료!


## 박민채

파생변수 생성

- current_avg_total: 평균 전류
- current_std_total: 표준편차 전류
- pf_avg_total: 평균 역률

- is_volatile_hour : 변동성 시간대인가? $0/1$ 컬럼
- is_low_efficiency_gear : 상위/중간/하위 그룹으로 Ordinary 변수

- total_current : 전류 합계 - 전체 부하의 크기 직관적으로 보여줌
- current_unbalance : 전류 불균형도 - 3상에서 전류가 얼마나 불균형하게 흐르는지 측정(설비의 이상 가동이나 부하 불균형을 포착)
- reactive_ratio : 무효전력이 전체 전류에서 차지하는 비중

In [22]:
import pandas as pd
import numpy as np
import gc

def create_final_ml_dataset_optimized(df, freq='1h'):
    # 필요한 컬럼만 추출 (view 사용으로 복사 최소화)
    required_cols = [
        'timestamp', 'module(equipment)', 'activePower', 'reactivePowerLagging',
        'currentR', 'currentS', 'currentT',
        'powerFactorR', 'powerFactorS', 'powerFactorT'
    ]

    # 집계 로직 설정
    agg_func = {
        'currentR': ['mean', 'std'], 'currentS': ['mean', 'std'], 'currentT': ['mean', 'std'],
        'powerFactorR': ['mean', 'std'], 'powerFactorS': ['mean', 'std'], 'powerFactorT': ['mean', 'std'],
        'reactivePowerLagging': 'mean',
        'activePower': 'mean'
    }

    modules = df['module(equipment)'].unique()
    all_results = []

    for mod in modules:
        print(f"🔄 처리 중: {mod} ...")

        # 1. 해당 설비 데이터만 추출 및 즉시 datetime 변환
        temp = df[df['module(equipment)'] == mod][required_cols].copy()
        temp['datetime'] = pd.to_datetime(temp['timestamp'], unit='ms')
        temp = temp.drop(columns=['timestamp']) # 메모리 확보

        # 2. 설비별 집계 실행
        res = temp.groupby(pd.Grouper(key='datetime', freq=freq)).agg(agg_func)
        res = res.reset_index()
        res['module'] = mod # 설비명 추가

        # 3. 파생변수 생성 (메모리가 적을 때 미리 계산)
        # [인사이트/물리 변수들]
        res['hour'] = res['datetime'].dt.hour
        res['is_volatile_hour'] = res['hour'].isin([0, 4, 10, 11, 12, 13, 15, 20, 21, 22]).astype(int)
        res['is_low_efficiency_gear'] = 0 if '15' in str(mod) else (1 if '17' in str(mod) else 2)

        curr_means = [('currentR', 'mean'), ('currentS', 'mean'), ('currentT', 'mean')]
        curr_stds = [('currentR', 'std'), ('currentS', 'std'), ('currentT', 'std')]
        pf_means = [('powerFactorR', 'mean'), ('powerFactorS', 'mean'), ('powerFactorT', 'mean')]

        res['current_avg_total'] = res[curr_means].mean(axis=1)
        res['current_std_total'] = res[curr_stds].mean(axis=1)
        res['pf_avg_total'] = res[pf_means].mean(axis=1)
        res['total_current'] = res[curr_means].sum(axis=1)
        res['current_unbalance'] = (res[curr_means].max(axis=1) - res[curr_means].min(axis=1)) / (res['current_avg_total'] + 1e-6)
        res['reactive_ratio'] = res[('reactivePowerLagging', 'mean')] / (res['total_current'] + 1e-6)
        res['weekday'] = res['datetime'].dt.weekday
        res['is_weekend'] = (res['weekday'] >= 5).astype(int)

        all_results.append(res)

        # 메모리 강제 정리
        del temp, res
        gc.collect()

    # 4. 모든 설비 데이터 합치기 및 컬럼명 정리
    final_res = pd.concat(all_results, axis=0).reset_index(drop=True)

    # 멀티인덱스 컬럼명 플래싱 (튜플 -> 문자열)
    final_res.columns = [
        f"{c[0]}_{c[1]}" if isinstance(c, tuple) and c[1] != "" else c[0]
        for c in final_res.columns
    ]

    return final_res

# 실행 (원본 train을 직접 넣지 말고 필요한 컬럼만 슬라이싱해서 전달)
ml_ready_data_1hour = create_final_ml_dataset_optimized(train)
print(f"✅ 데이터셋 생성 완료! 행 수: {len(ml_ready_data_1hour)}")

🔄 처리 중: 1(PM-3) ...
🔄 처리 중: 11(우측분전반1) ...
🔄 처리 중: 12(4호기) ...
🔄 처리 중: 13(3호기) ...
🔄 처리 중: 14(2호기) ...
🔄 처리 중: 15(예비건조기) ...
🔄 처리 중: 16(호이스트) ...
🔄 처리 중: 17(6호기) ...
🔄 처리 중: 18(우측분전반2) ...
🔄 처리 중: 2(L-1전등) ...
🔄 처리 중: 3(분쇄기(2)) ...
🔄 처리 중: 4(분쇄기(1)) ...
🔄 처리 중: 5(좌측분전반) ...
✅ 데이터셋 생성 완료! 행 수: 37739
